In [5]:
pip install opencv-python-headless scikit-learn joblib pandas

   ---------------------------------------- 0.0/39.4 MB ? eta -:--:--
   - -------------------------------------- 1.6/39.4 MB 7.6 MB/s eta 0:00:05
   --- ------------------------------------ 3.4/39.4 MB 8.4 MB/s eta 0:00:05
   ----- ---------------------------------- 5.5/39.4 MB 8.8 MB/s eta 0:00:04
   ------- -------------------------------- 7.3/39.4 MB 8.9 MB/s eta 0:00:04
   ---------- ----------------------------- 10.0/39.4 MB 9.4 MB/s eta 0:00:04
   ------------ --------------------------- 12.3/39.4 MB 9.9 MB/s eta 0:00:03
   -------------- ------------------------- 14.7/39.4 MB 10.1 MB/s eta 0:00:03
   ----------------- ---------------------- 17.3/39.4 MB 10.3 MB/s eta 0:00:03
   ------------------- -------------------- 19.7/39.4 MB 10.5 MB/s eta 0:00:02
   ---------------------- ----------------- 22.3/39.4 MB 10.7 MB/s eta 0:00:02
   ------------------------- -------------- 25.2/39.4 MB 10.8 MB/s eta 0:00:02
   ---------------------------- ----------- 27.8/39.4 MB 11.0 MB/s eta 

In [6]:
import os
import cv2
import numpy as np
import joblib
from sklearn.svm import OneClassSVM
from sklearn.decomposition import PCA
from concurrent.futures import ThreadPoolExecutor
import pandas as pd


In [7]:
IMAGE_SIZE = (128, 128)  # Resize images to this size
N_COMPONENTS = 500       # PCA components
NU_VALUE = 0.02          # SVM sensitivity (lower = stricter detection)

In [8]:
def load_and_preprocess_image(image_path):
    try:
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"❌ Warning: Unable to read {image_path}")
            return None
        img = cv2.resize(img, IMAGE_SIZE)
        img = img.astype(np.float32) / 255.0
        return img.flatten()
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None


In [9]:
def load_images_from_folder(folder):
    images = []
    filenames = []

    if not os.path.exists(folder):
        print(f"❌ Error: Folder '{folder}' not found!")
        return np.array(images), filenames

    file_list = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith((".png", ".jpg"))]

    if not file_list:
        print(f"⚠️ Warning: No images found in '{folder}'")
        return np.array(images), filenames

    with ThreadPoolExecutor() as executor:
        results = list(executor.map(load_and_preprocess_image, file_list))

    for i, result in enumerate(results):
        if result is not None:
            images.append(result)
            filenames.append(os.path.basename(file_list[i]))

    return np.array(images), filenames


In [10]:
def train_one_class_svm(normal_folder, model_path="one_class_svm_model.pkl"):
    print("\n📂 Loading normal images for training...")
    X_train, _ = load_images_from_folder(normal_folder)

    if X_train.shape[0] == 0:
        print("❌ Error: No training images found!")
        return

    print(f"✅ Loaded {X_train.shape[0]} normal images for training.")
    print(f"Feature vector size: {X_train.shape[1]}")

    print("\n🧠 Applying PCA for feature reduction...")
    pca = PCA(n_components=min(N_COMPONENTS, X_train.shape[1]))
    X_train_pca = pca.fit_transform(X_train)

    print("\n🚀 Training One-Class SVM...")
    svm_model = OneClassSVM(kernel="rbf", gamma="auto", nu=NU_VALUE)
    svm_model.fit(X_train_pca)

    joblib.dump((svm_model, pca), model_path)
    print(f"✅ Model trained and saved at {model_path}")


In [11]:
def test_one_class_svm(test_folder, model_path="one_class_svm_model.pkl", output_csv="anomaly_results.csv"):
    if not os.path.exists(model_path):
        print("❌ Error: Model file not found! Train the model first.")
        return

    print("\n📂 Loading test images...")
    X_test, filenames = load_images_from_folder(test_folder)

    if X_test.shape[0] == 0:
        print("❌ No test images found!")
        return

    print(f"✅ Loaded {X_test.shape[0]} test images.")

    svm_model, pca = joblib.load(model_path)
    X_test_pca = pca.transform(X_test)

    print("\n🔍 Predicting anomalies...")
    predictions = svm_model.predict(X_test_pca)

    results = pd.DataFrame({"Filename": filenames, "Prediction": predictions})
    results["Prediction"] = results["Prediction"].map({1: "Normal ✅", -1: "Anomaly ❌"})
    results.to_csv(output_csv, index=False)

    print(f"\n📄 Results saved to {output_csv}")
    print("\n🔹 **Anomaly Detection Summary** 🔹")
    print(results.value_counts("Prediction"))
    print("\n✅ Detection completed!")


In [15]:
# Set your folder paths
normal_data_folder = r"C:\Users\SIL-Laptop\Desktop\LSB CNN MMS\One_Class_SVM\Normal"    # Folder with normal images
test_data_folder = r"C:\Users\SIL-Laptop\Desktop\LSB CNN MMS\One_Class_SVM\Attack"      # Folder with test images (normal + anomalies)
svm_model_path = r"C:\Users\SIL-Laptop\Desktop\LSB CNN MMS\One_Class_SVM\one_class_svm_model(new).pkl"
output_results_csv = r"C:\Users\SIL-Laptop\Desktop\LSB CNN MMS\One_Class_SVM\anomaly_results(new).csv"

# Train the model
train_one_class_svm(normal_data_folder, svm_model_path)

# Test the model
test_one_class_svm(test_data_folder, svm_model_path, output_results_csv)



📂 Loading normal images for training...
✅ Loaded 4732 normal images for training.
Feature vector size: 16384

🧠 Applying PCA for feature reduction...

🚀 Training One-Class SVM...
✅ Model trained and saved at C:\Users\SIL-Laptop\Desktop\LSB CNN MMS\One_Class_SVM\one_class_svm_model(new).pkl

📂 Loading test images...
✅ Loaded 4756 test images.

🔍 Predicting anomalies...

📄 Results saved to C:\Users\SIL-Laptop\Desktop\LSB CNN MMS\One_Class_SVM\anomaly_results(new).csv

🔹 **Anomaly Detection Summary** 🔹
Prediction
Normal ✅    4756
Name: count, dtype: int64

✅ Detection completed!
